# Protein Hits Count in *Candidatus Electrothrix communis RB*

This is the same notebook I used for to compute the protein hits for the Kalø Vig and Løgten samples but adapted to the *Candidatus Electrothrix communis RB* sample. For further information (for example, about multiple hits) please refer to the original notebook `protein_hits.ipynb`.

In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
from Bio import SeqIO

In [2]:
# Open BLAST output tables
marine_gs_illumina_blast = pd.read_table(
    "genes/results/2022-05-19/blast/marine_gs_illumina/marine_gs_illumina_omc_blast.tsv",
    header=None,
)

# Drop the duplicated protein sequence-contig pairs with the highest e-value
marine_gs_illumina_blast = marine_gs_illumina_blast.sort_values(10).drop_duplicates(
    subset=[0, 1]
)

# Select the first two columns
marine_gs_illumina_blast = marine_gs_illumina_blast.iloc[:, [0, 1, 10]]

# Add column names
marine_gs_illumina_blast.columns = ["protein_id", "contig", "e-value"]

What may happen is that we have the exactly same protein sequence (with the underscore) to appear in two different query protein sequences. This will confuse the results as when we expect just one hit for that protein sequence, there may be two in two different query protein sequences.

For example, if we have the exactly same protein sequence to appear in two different protein, it may seem like we have two hits in two different protein (row 2) while in reality, we only have one hit in one protein (row 1).

|      | Protein 1 | Protein 2 | Correct |
|------|-----------|-----------|---------|
| bin1 | 1         | 0         | True    |
| bin1 | 1         | 1         | False   |

Thus, I will remove the duplicated contigs (protein sequences) from the BLAST output beforehand keeping the ones with the lowest e-value.

In [3]:
def remove_contig_dups(df):
    """
    Remove the duplicated protein sequences (inside contigs) with the highest e-value from BLAST output.

    Parameters
    ----------
    df: pd.DataFrame
        Input dataframe, should have e-value and contig columns
    Returns
    -------
    pd.DataFrame
    """
    return df.sort_values("e-value").drop_duplicates(subset=["contig"])

marine_gs_illumina_blast = remove_contig_dups(marine_gs_illumina_blast)

Create column names (protein names) and indices (bin names) for the final dataframes.

In [5]:
# Protein accession numbers are identical in both BLAST outputs
protein_col_names = marine_gs_illumina_blast["protein_id"].unique()


def get_bins(sample_path):
    """Extract bin names without extension."""
    index = []
    for file in os.listdir(sample_path):
        index.append(Path(file).stem)
    return index


marine_gs_illumina_bins = get_bins(
    "genes/results/2022-05-19/prodigal/marine_gs_illumina"
)
marine_gs_nanopore_bins = get_bins(
    "genes/results/2022-05-19/prodigal/marine_gs_nanopore"
)

In [6]:
def contig_hits(sample_name, blast_output, bin_path, bins):
    """
    Save a dataframe with the number of sequence hits of each protein in each metagenomic bin.
    """

    # Final dataframe skeleton
    final_df = pd.DataFrame(0, index=bins, columns=protein_col_names)

    for bin_file in Path(bin_path + sample_name).iterdir():

        # Save the current bin's sequence ids to a dataframe
        bin_record_ids = []
        try:
            for record in SeqIO.parse(bin_file, "fasta"):
                bin_record_ids.append(record.id)
            bin_record_ids_df = pd.DataFrame(bin_record_ids, columns=["contig"])
        except UnicodeDecodeError:
            continue

        # Merge the dataframe of sequence ids of the current bin and blast output
        merged = bin_record_ids_df.merge(blast_output, on="contig")

        # Group the merge dataframe to compute the total number of contig hits for the protein in the bin
        grouped = merged.groupby("protein_id").count().reset_index()

        # The name of the current bin
        current_bin = Path(bin_file).stem

        # Loop through the grouped dataframe and extract protein id and number of contig hits
        # Then add the number to the corresponding row/column of the dataset
        for index, row in grouped.iterrows():
            final_df.loc[current_bin, row["protein_id"]] = row["contig"]

    return final_df

In [7]:
# Coount protein contig hits
marine_gs_illumina_contig_hits = contig_hits(
    sample_name="marine_gs_illumina",
    blast_output=marine_gs_illumina_blast,
    bin_path="genes/results/2022-05-19/prodigal/",
    bins=marine_gs_illumina_bins,
)

marine_gs_nanopore_contig_hits = contig_hits(
    sample_name="marine_gs_nanopore",
    blast_output=marine_gs_nanopore_blast,
    bin_path="genes/results/2022-05-19/prodigal/",
    bins=marine_gs_nanopore_bins,
)

# Remove rows with all 0s
marine_gs_illumina_contig_hits = marine_gs_illumina_contig_hits.loc[(marine_gs_illumina_contig_hits!=0).any(axis=1)]
marine_gs_nanopore_contig_hits = marine_gs_nanopore_contig_hits.loc[(marine_gs_nanopore_contig_hits!=0).any(axis=1)]

# Save to csv
marine_gs_illumina_contig_hits.to_csv("genes/results/2022-05-20/blast/marine_gs_illumina_contig_hits.csv")
marine_gs_nanopore_contig_hits.to_csv("genes/results/2022-05-20/blast/marine_gs_nanopore_contig_hits.csv")